In [597]:
#importing everything before getting started
import pandas as pd
import openai
import numpy as np

#used to embed, and then to search 
from openai.embeddings_utils import get_embedding, cosine_similarity

#this is the embedding model used in the program
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"
max_tokens = 8000

In [598]:
#loading and printing dataset
#pd.options.display.max_colwidth = 500 | if csv not printing probably, uncomment this line and set it equal to the max length
df = pd.read_csv("info.csv")
df

,Information,Category: Paragraph – Page Heading,"Link: Provided, Source"
0,The CS Placement Exam opens on May 5th! It will close on December 31st. Visit our FAQ webpage for more information.,Advising Announcements – Advising,"https://cs.ucdavis.edu/undergraduate/current-majors/frequently-asked-questions, https://cs.ucdavis.edu/advising"
1,The Winter 2024 and Spring 2024 tentative course offerings are now viewable!,Advising Announcements – Advising,"https://cs.ucdavis.edu/course-schedule, https://cs.ucdavis.edu/advising"
2,Need help in your ECS courses? Check out CS Tutoring (peer-run service) at UC Davis!,Advising Announcements – Advising,"https://sites.google.com/view/cs-tutoring-ucd/, https://cs.ucdavis.edu/advising"
3,Need a place to study on campus? Take a look at some of the study spaces available on the Student Affairs website.,Advising Announcements – Advising,https://studentaffairs.ucdavis.edu/getting-help/academics/study-materials
4,Student Computer Recommendations,Advising Announcements – Advising,"https://engineering.ucdavis.edu/undergraduates/academic-advising/student-computer-recommendations, https://cs.ucdavis.edu/advising"
5,UC Davis Student Resources – FAQ’s,Advising Announcements – Advising,"https://ebeler.faculty.ucdavis.edu/resources/faq-student-resources/, https://cs.ucdavis.edu/advising"
6,The Computer Science (CS) Undergraduate Advising team provides major academic advising to currently matriculated UCD undergraduate students in Computer Science (CS) and Computer Science & Engineering (CSE).,Who We Are (P1) – Advising,https://cs.ucdavis.edu/advising
7,CS Undergraduate Advisors partner with CS and CSE students to provide assistance on their academic journey. Our goal is to teach our students how to understand their academic requirements and navigate the university resources available to help them along the way.,Who We Are (P2) – Advising,https://cs.ucdavis.edu/advising
8,Advising is a collaborative process that will require each student to play an active role.,Who We Are (P2) – Advising,https://cs.ucdavis.edu/advising
9,"Appointments are available to students declared in the CS and CSE majors only. If you are interested in changing majors, please visit our Change of Major webpage.",Services→ Appointments – Advising,"https://cs.ucdavis.edu/undergraduate/non-majors/changing-majors-double-majors, https://cs.ucdavis.edu/advising"


In [599]:
# OPENAI API Key
openai.api_key="YOUR-KEY"

In [600]:
# creating embeddings 

#calling embedding model
embedding_model = "text-embedding-ada-002"

#creating a new column in DataFrame titled embedding that uses model's embedding function to create embeddings
df['embedding'] = df['Information'].apply(lambda x: get_embedding(x, engine = embedding_model))

#creating a new csv file and DataFrame for embedded data
df.to_csv("updated_info.csv")
df_updated = pd.read_csv("updated_info.csv")
df_updated

# reshaping embedding data from string to array
df_updated['embedding'] = df_updated['embedding'].apply(literal_eval).apply(np.array)

In [601]:
# commented
# creating a search function to determine what information needs to be retrieved
# parameters are : cleaned datasets, user query, number of returned results, and bool to print
def search_sort(df_updated, question, n=2, pprint=True):
    
    # taking in user data and creating its embedding
    question_embedding = get_embedding(question, engine='text-embedding-ada-002')
    
    #creating a similarity column to determine relevance of information line to query
    df_updated["similarities"] = df_updated.embedding.apply(lambda x: cosine_similarity(x, question_embedding))
    
    #sorting according to most relevant/similiar
    results = df_updated.sort_values("similarities", ascending=False).head(n)
    
    #printing results
    for r,y,z in zip(results['Information'], results['Category: Paragraph – Page Heading'], results['Link: Provided, Source']):
        print("Result: ", r, " | ", y, " | ", z)
        print()
    
    return results


In [602]:
# first call of function
res = search_sort(df_updated, "student affairs", n=2)

Result:  Need a place to study on campus? Take a look at some of the study spaces available on the Student Affairs website.  |  Advising Announcements – Advising  |  https://studentaffairs.ucdavis.edu/getting-help/academics/study-materials

Result:  Jason Sison – Undergraduate Academic Advisor  |  CS Advising Team - Advising  |  https://cs.ucdavis.edu/directory/jason-sison



In [603]:
# second call of function
res = search_sort(df_updated, "advisor", n=4)

Result:  Andrea Abrahamson – Undergraduate Academic Advisor  |  CS Advising Team - Advising  |  https://cs.ucdavis.edu/directory/andrea-abrahamson

Result:  Jason Sison – Undergraduate Academic Advisor  |  CS Advising Team - Advising  |  https://cs.ucdavis.edu/directory/jason-sison

Result:  Advising is a collaborative process that will require each student to play an active role.  |  Who We Are (P2) – Advising  |  https://cs.ucdavis.edu/advising

Result:  CS Undergraduate Advisors partner with CS and CSE students to provide assistance on their academic journey. Our goal is to teach our students how to understand their academic requirements and navigate the university resources available to help them along the way.   |  Who We Are (P2) – Advising  |  https://cs.ucdavis.edu/advising



In [604]:
# third call of function
res = search_sort(df_updated, "change of major", n=5)

Result:  For more information on requirements to change to CS or CSE, please review our Change of Major webpage.  |  Change of Major Advising Sessions – Advising  |  https://cs.ucdavis.edu/undergraduate/non-majors/changing-majors-double-majors, https://cs.ucdavis.edu/advising

Result:  If you are interested in changing to one of our majors, please attend a Change of Major Info Session. You can view the upcoming dates and register on our Changing Majors webpage.  |  Services → Drop-in Hours (Via Zoom) – Advising  |  https://cs.ucdavis.edu/undergraduate/non-majors/changing-majors-double-majors, https://cs.ucdavis.edu/advising

Result:  Appointments are available to students declared in the CS and CSE majors only. If you are interested in changing majors, please visit our Change of Major webpage.  |  Services→ Appointments – Advising  |  https://cs.ucdavis.edu/undergraduate/non-majors/changing-majors-double-majors, https://cs.ucdavis.edu/advising

Result:  Students interested in changing 